<a href="https://colab.research.google.com/github/tristanpadiou/Travel-assistant/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
! pip install langchain_google_genai
! pip install langchain-community
! pip install langgraph
! pip install pyowm
! pip install wikipedia
! pip install gradio


In [109]:
import pandas as pd
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import initialize_agent, load_tools
from langchain.tools import Tool,tool
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.schema import AIMessage, HumanMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.messages import (
    SystemMessage,
    HumanMessage,
)
#get graph visuals
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles

from typing_extensions import TypedDict
from typing import Annotated
import pytz
from datetime import datetime
import os
import requests
import json
import gradio as gr


In [3]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
pse=userdata.get('programmable_search_engine')
OPENWEATHERMAP_API_KEY=userdata.get('open_weather_key')
os.environ['OPENWEATHERMAP_API_KEY']=OPENWEATHERMAP_API_KEY

In [4]:
GEMINI_MODEL='gemini-1.5-flash'

In [5]:
llm = ChatGoogleGenerativeAI(google_api_key=GOOGLE_API_KEY, model=GEMINI_MODEL, temperature=0.3)

In [104]:
# trip data

schedule = {
    "Jeudi 26 Décembre": [
        {"time": "09:00", "event": "Petit Déjeuner"},
        {"time": "10:00", "event": "Atelier de Tissage", "details": "1 personne"},
        {"time": "16:00", "event": "Visite guidée de la Koutoubia"},
        {"time": "19:00", "event": "Vol EJU4665 Paris", "details": "2 personnes /Couple Sekfali"},
        {"time": "20:30", "event": "Dîner au riad", "details": "12 ou 13 personnes"},
    ],
    "Vendredi 27 Décembre": [
        {"time": "09:00", "event": "Petit Déjeuner"},
        {"time": "10:20", "event": "Vol LX2200 Genève", "details": "1 personne William Burgat"},
        {"time": "10:00", "event": "Visite Guidée des Tombeaux Saadiens"},
        {"time": "15:00", "event": "Visite aux Jardins de Majorelle et Musée YSL"},
        {"time": "19:00", "event": "Transfert pour aller au restaurant AZAR"},
        {"time": "19:30", "event": "Dîner au restaurant AZAR"},
    ],
    "Samedi 28 Décembre": [
        {"time": "08:00", "event": "Petit Déjeuner"},
        {"time": "09:00", "event": "Randonnée dans l'Atlas"},
        {"time": "19:30", "event": "Dîner au Riad", "details": "11 personnes"},
        {"time": "21:00", "event": "Soirée KARAOKE"},
    ],
    "Dimanche 29 Décembre": [
        {"time": "09:00", "event": "Petit Déjeuner"},
        {"time": "09:30", "event": "Arrivée de Lili-Chambre Jade"},
        {"time": "14:30", "event": "Excursion en Quads", "details": "(2 heures)"},
        {"time": "Dîner", "event": "Dîner au Scarabeo Camp"},
    ],
    "Lundi 30 Décembre": [
        {"time": "07:00", "event": "Petit Déjeuner"},
        {"time": "08:00", "event": "Excursion à Ouarzazate"},
        {"time": "20:00", "event": "Dîner au Riad", "details": "12 personnes"},
    ],
    "Mardi 31 Décembre": [
        {"time": "09:00", "event": "Petit Déjeuner"},
        {"time": "10:00", "event": "Visite Guidée à la Merdersa Ben Youssef"},
        {"time": "14:00", "event": "HAMAM et MASSAGE x 2 à vérifier"},
        {"time": "15:30", "event": "HAMAM et MASSAGE x 2 à vérifier"},
        {"time": "17:00", "event": "HAMAM et MASSAGE x 2 à vérifier"},
        {"time": "17:00", "event": "Coiffeuse et vernis"},
        {"time": "20:00", "event": "Réveillon au Riad avec DJ"},
    ],
    "Mercredi 1er Janvier": [
        {"time": "12:00", "event": "HAMAM et MASSAGE à vérifier"},
        {"time": "13:30", "event": "HAMAM et MASSAGE à vérifier"},
        {"time": "15:00", "event": "HAMAM et MASSAGE à vérifier"},
        {"time": "16:15", "event": "Excursion en Sidecar", "details": "(1h30)"},
        {"time": "19:30", "event": "Dîner au Restaurant Dar Yacout"},
    ],
    "Jeudi 2 Janvier": [
        {"time": "11:00", "event": "Atelier de cuisine"},
        {"time": "14:00", "event": "Déjeuner à la terrasse"},
        {"time": "16:30", "event": "Départ Sekfali x 2"},
        {"time": "Programme à définir", "event": "Programme à définir"},
    ],
    "Vendredi Janvier": [
        {"time": "08:00", "event": "Départ famille Padiou x 8"},
        {"time": "09:00", "event": "Départ Rossignol x 2"},
    ],
}



In [78]:
# location
loc='Marrakech'
hotel='le clos des arts'


# initializing time and date tool

def time_tool(query: str):
  """
  tool to get the current time in a city.
  Args:'continent/city'

  """

  timezone = pytz.timezone(query)
  # Get the current time in UTC, and then convert it to the Marrakech timezone
  utc_now = datetime.now(pytz.utc)  # Get current time in UTC
  localized_time = utc_now.astimezone(timezone)  # Convert to Marrakech time
  time=localized_time.strftime('%Y-%m-%d %H:%M:%S')
  return time

current_time_tool=Tool(name='current_time_tool', func=time_tool, description='To get the current time in any city, the input should be a location string (eg."Continent/City") ')


In [54]:
prompt_template="""
You are provided with the following data:

location: {loc}
The hotel: {hotel}
Today's date: {date}
The schedule: {data}

Please answer the following question:
{question}
"""

In [9]:
def google_image_search(query: str) -> str:
  """Search for images using Google Custom Search API"""
  # Define the API endpoint for Google Custom Search
  url = "https://www.googleapis.com/customsearch/v1"

  params = {
      "q": query,
      "cx": pse,
      "key": GOOGLE_API_KEY,
      "searchType": "image",  # Search for images
      "num": 1  # Number of results to fetch
  }

  # Make the request to the Google Custom Search API
  response = requests.get(url, params=params)
  data = response.json()

  # Check if the response contains image results
  if 'items' in data:
      # Extract the first image result
      image_url = data['items'][0]['link']
      return f"Here is an image related to your query: {image_url}"
  else:
      return "Sorry, no images were found for your query."

google_image_tool=Tool(name='google_image_tool', func=google_image_search, description='Use this tool to search for images using Google Custom Search API')

In [127]:
# langgraph
#loading tools
api_tools=load_tools(['openweathermap-api','wikipedia'])
langgraph_tools=[current_time_tool,google_image_tool]+api_tools


# state
class State(TypedDict):
    messages: Annotated[list, add_messages]



graph_builder = StateGraph(State)

# Modification: tell the LLM which tools it can call
llm_with_tools = llm.bind_tools(langgraph_tools)


def chatbot(state: State):
  """You are a travel assistant that answers user questions about their trip. Depending on the request, leverage which tools to use if necessary."""
  return {"messages": [llm_with_tools.invoke(state["messages"])]}

def get_schedule(state: State):
  """ get the complete schedule of the trip"""
  messages = [
    SystemMessage(content=schedule),
    HumanMessage(content=state["messages"]),
]
  return state["schedule"]=schedule


graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=langgraph_tools)
graph_builder.add_node("tools", tool_node)
graph_builder.add_node('get_schedule',get_schedule)

# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.set_entry_point("get_schedule")
graph_builder.add_edge("get_schedule", "chatbot")
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
graph = graph_builder.compile()



display(
    Image(
        graph.get_graph().draw_mermaid_png(
            draw_method=MermaidDrawMethod.API,
        )
    )
)



SyntaxError: invalid syntax (<ipython-input-127-b99a58876f66>, line 25)

In [126]:
State['schedule']=schedule

TypeError: '_TypedDictMeta' object does not support item assignment

In [123]:
message["schedule"]=schedule

In [124]:
message['schedule']

{'Jeudi 26 Décembre': [{'time': '09:00', 'event': 'Petit Déjeuner'},
  {'time': '10:00', 'event': 'Atelier de Tissage', 'details': '1 personne'},
  {'time': '16:00', 'event': 'Visite guidée de la Koutoubia'},
  {'time': '19:00',
   'event': 'Vol EJU4665 Paris',
   'details': '2 personnes /Couple Sekfali'},
  {'time': '20:30',
   'event': 'Dîner au riad',
   'details': '12 ou 13 personnes'}],
 'Vendredi 27 Décembre': [{'time': '09:00', 'event': 'Petit Déjeuner'},
  {'time': '10:20',
   'event': 'Vol LX2200 Genève',
   'details': '1 personne William Burgat'},
  {'time': '10:00', 'event': 'Visite Guidée des Tombeaux Saadiens'},
  {'time': '15:00', 'event': 'Visite aux Jardins de Majorelle et Musée YSL'},
  {'time': '19:00', 'event': 'Transfert pour aller au restaurant AZAR'},
  {'time': '19:30', 'event': 'Dîner au restaurant AZAR'}],
 'Samedi 28 Décembre': [{'time': '08:00', 'event': 'Petit Déjeuner'},
  {'time': '09:00', 'event': "Randonnée dans l'Atlas"},
  {'time': '19:30', 'event': 'D

In [113]:
def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [("user", user_input)]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)

stream_graph_updates('what is the schedule')

TypeError: 'NoneType' object is not subscriptable

In [ ]:
interface=gr.ChatInterface(
    manager_agent,
    type='messages'
)
interface.launch(share=True,debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b8df5473c2d71257bc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


<ipython-input-12-d0c598df56b5>:9: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. See LangGraph documentation for more details: https://langchain-ai.github.io/langgraph/. Refer here for its pre-built ReAct agent: https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/
  tools_agent = initialize_agent(tools,




> Entering new AgentExecutor chain...
Thought:To answer the question about fun facts about the Koutoubia, I need to use the wikipedia tool to search for information about the Koutoubia Mosque.

Action: wikipedia
Action Input: Koutoubia Mosque

Observation: Page: Kutubiyya Mosque
Summary: The Kutubiyya Mosque or Koutoubia Mosque (Arabic: جامع الكتبية Arabic pronunciation: [jaːmiʕu‿lkutubijːa(h)]) is the largest mosque in Marrakesh, Morocco. It is located in the southwest medina quarter of Marrakesh, near the Jemaa el-Fnaa market place, and is flanked by large gardens.
The mosque was founded in 1147 by the Almohad caliph Abd al-Mu'min right after he conquered Marrakesh from the Almoravids. A second version of the mosque was entirely rebuilt by Abd al-Mu'min around 1158, with Ya'qub al-Mansur possibly finalizing construction of the minaret around 1195. This second mosque is the structure that stands today. It is an important example of Almohad architecture and of Moroccan mosque archite

<ipython-input-13-433f5a715cd1>:7: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm, prompt=prompt)




> Entering new AgentExecutor chain...
Thought:I need to find the construction date of the Koutoubia Mosque.  I can use the wikipedia tool for this.

Action: wikipedia
Action Input: Koutoubia Mosque

Observation: Page: Kutubiyya Mosque
Summary: The Kutubiyya Mosque or Koutoubia Mosque (Arabic: جامع الكتبية Arabic pronunciation: [jaːmiʕu‿lkutubijːa(h)]) is the largest mosque in Marrakesh, Morocco. It is located in the southwest medina quarter of Marrakesh, near the Jemaa el-Fnaa market place, and is flanked by large gardens.
The mosque was founded in 1147 by the Almohad caliph Abd al-Mu'min right after he conquered Marrakesh from the Almoravids. A second version of the mosque was entirely rebuilt by Abd al-Mu'min around 1158, with Ya'qub al-Mansur possibly finalizing construction of the minaret around 1195. This second mosque is the structure that stands today. It is an important example of Almohad architecture and of Moroccan mosque architecture generally. The minaret tower, 77 metres

In [ ]:
print(f"Gradio instance URL: {url[2]}")